<h2> Criando o conjunto de dados e importando


In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


In [85]:
path = "../Atividades/Atividade 6/OnlineRetail.xlsx"
df = pd.read_excel(path)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


<h2> 1. Inicialmente, faça uma preparação dos dados, sendo:


<h3>  Remover espaços em branco do começo e fim da Descrição


In [86]:
df['Description'] = df['Description'].str.strip()
df['Description']

0          WHITE HANGING HEART T-LIGHT HOLDER
1                         WHITE METAL LANTERN
2              CREAM CUPID HEARTS COAT HANGER
3         KNITTED UNION FLAG HOT WATER BOTTLE
4              RED WOOLLY HOTTIE WHITE HEART.
                         ...                 
541904            PACK OF 20 SPACEBOY NAPKINS
541905            CHILDREN'S APRON DOLLY GIRL
541906           CHILDRENS CUTLERY DOLLY GIRL
541907        CHILDRENS CUTLERY CIRCUS PARADE
541908           BAKING SET 9 PIECE RETROSPOT
Name: Description, Length: 541909, dtype: object

<h3>   Remover linhas que não possuem InvoiceNo


In [87]:
df.dropna(axis=0,subset=['InvoiceNo'],inplace=True)
df.InvoiceNo.isna().unique()

array([False])

<h3>  Converter InvoiceNo para string e remover os que começam
com C (cancelamento)



In [88]:
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[df['Quantity'] > 0]
df = df[~df["InvoiceNo"].str.contains('C')]
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


<h2>Converter os dados para o formato de transações, já filtrando
apenas para o país "France":

In [40]:
df = df[df['Country'] == 'France']
df_grouped = df.groupby(['InvoiceNo','Description'])['Quantity'].sum()

cestaFranca = df_grouped.unstack(fill_value=0)
cestaFranca = cestaFranca.reset_index().set_index('InvoiceNo')
cestaFranca

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580986,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581171,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h2> Formatar a cesta, alterando valores maiores que 1 (que
representam a quantidade) em 1, e valores menores que 0 (caso
ocorram) em 0. Também devemos remover a coluna ‘POSTAGE’
que representa o envio, e não um produto.

In [135]:
def encode_units(x):
    if x <= 0:
        return False
    else:
        return True
cestaFranca = cestaFranca.applymap(encode_units)
#cestaFranca.drop('POSTAGE',inplace=True,axis=1)
cestaFranca

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536852,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536974,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
537065,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
537463,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580986,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
581001,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
581171,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [136]:
cestaFranca[cestaFranca["ZINC HERB GARDEN CONTAINER"] == True]

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
552826,False,False,False,False,False,False,False,False,True,False,...,False,True,False,False,False,False,True,False,False,False


<h2> Calcule os itemsets frequentes e as regras de associação, e
responda às seguintes perguntas (em relação às compras feitas
na França apenas):<h2> 

In [138]:
frequent_itemsets = apriori(cestaFranca, min_support=0.01,use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.030612,(10 COLOUR SPACEBOY PEN)
1,0.015306,(12 COLOURED PARTY BALLOONS)
2,0.015306,(12 PENCIL SMALL TUBE WOODLAND)
3,0.017857,(12 PENCILS SMALL TUBE RED RETROSPOT)
4,0.017857,(12 PENCILS SMALL TUBE SKULL)
...,...,...
29856,0.010204,"(LUNCH BOX WITH CUTLERY RETROSPOT, ALARM CLOCK..."
29857,0.010204,"(LUNCH BOX WITH CUTLERY RETROSPOT, ALARM CLOCK..."
29858,0.010204,"(LUNCH BOX WITH CUTLERY RETROSPOT, LUNCH BAG A..."
29859,0.010204,"(LUNCH BOX WITH CUTLERY RETROSPOT, ALARM CLOCK..."


In [139]:
regras = association_rules(frequent_itemsets,metric="confidence", min_threshold=0.7)
regras

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(12 COLOURED PARTY BALLOONS),(SET OF 9 BLACK SKULL BALLOONS),0.015306,0.066327,0.012755,0.833333,12.564103,0.011740,5.602041
1,(12 PENCILS TALL TUBE WOODLAND),(12 PENCILS TALL TUBE RED RETROSPOT),0.015306,0.022959,0.012755,0.833333,36.296296,0.012404,5.862245
2,(36 PENCILS TUBE WOODLAND),(36 PENCILS TUBE RED RETROSPOT),0.022959,0.045918,0.020408,0.888889,19.358025,0.019354,8.586735
3,(BINGO SET),(4 TRADITIONAL SPINNING TOPS),0.010204,0.071429,0.010204,1.000000,14.000000,0.009475,inf
4,(6 GIFT TAGS 50'S CHRISTMAS),(6 GIFT TAGS VINTAGE CHRISTMAS),0.017857,0.025510,0.012755,0.714286,28.000000,0.012300,3.410714
...,...,...,...,...,...,...,...,...,...
675001,"(ALARM CLOCK BAKELIKE GREEN, DOLLY GIRL CHILDR...","(LUNCH BOX WITH CUTLERY RETROSPOT, ALARM CLOCK...",0.012755,0.010204,0.010204,0.800000,78.400000,0.010074,4.948980
675002,"(ALARM CLOCK BAKELIKE GREEN, JUMBO BAG APPLES)","(LUNCH BOX WITH CUTLERY RETROSPOT, ALARM CLOCK...",0.012755,0.010204,0.010204,0.800000,78.400000,0.010074,4.948980
675003,"(DOLLY GIRL CHILDRENS CUP, ALARM CLOCK BAKELIK...","(LUNCH BOX WITH CUTLERY RETROSPOT, ALARM CLOCK...",0.012755,0.010204,0.010204,0.800000,78.400000,0.010074,4.948980
675004,"(JUMBO BAG APPLES, ALARM CLOCK BAKELIKE RED)","(LUNCH BOX WITH CUTLERY RETROSPOT, ALARM CLOCK...",0.012755,0.010204,0.010204,0.800000,78.400000,0.010074,4.948980


<h3> a) Qual a regra com maior suporte?


In [140]:
frequent_itemsets.sort_values(by='support',ascending=False).head(1)

,support,itemsets
331,0.188776,(RABBIT NIGHT LIGHT)


<p> A regra com maios suporte é do seguinte itemset: (RABBIT NIGHT LIGHT)

<h3> b) Qual a regra com maior confiança?


In [141]:
regras.sort_values(by='confidence',ascending=False).head(1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
675005,"(DOLLY GIRL CHILDRENS CUP, JUMBO BAG APPLES)","(LUNCH BOX WITH CUTLERY RETROSPOT, ALARM CLOCK...",0.010204,0.010204,0.010204,1.0,98.0,0.0101,inf


<h3>c) Qual a regra com maior lift?


In [142]:
regras.sort_values(by='lift',ascending=False).head(1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
675005,"(DOLLY GIRL CHILDRENS CUP, JUMBO BAG APPLES)","(LUNCH BOX WITH CUTLERY RETROSPOT, ALARM CLOCK...",0.010204,0.010204,0.010204,1.0,98.0,0.0101,inf


<h3> d) Quantas regras possuem suporte maior ou igual a 0.02 e
confiança maior ou igual a 0.8?

In [143]:
len(regras[(regras["support"] >= 0.02) & (regras["confidence"] >= 0.8)])

928

<h3> e) Quantas regras possuem lift maior ou igual a 4 e confiança
maior ou igual a 0.8?

In [144]:
len(regras[(regras["lift"] >= 4) & (regras["confidence"] >= 0.8)])

552336

<h3> f) Quais produtos você recomendaria colocar próximos se você
fosse organizar um supermercado físico?

In [145]:
regras[(regras["lift"] >= 4) & (regras["confidence"] >= 0.8)].sort_values(by='support',ascending=False).head(30)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
233,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.137755,0.127551,0.122449,0.888889,6.968889,0.104878,7.852041
234,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.127551,0.137755,0.122449,0.960000,6.968889,0.104878,21.556122
228,(SET/6 RED SPOTTY PAPER PLATES),(SET/20 RED RETROSPOT PAPER NAPKINS),0.127551,0.132653,0.102041,0.800000,6.030769,0.085121,4.336735
7298,"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...",(SET/6 RED SPOTTY PAPER PLATES),0.102041,0.127551,0.099490,0.975000,7.644000,0.086474,34.897959
7300,"(SET/6 RED SPOTTY PAPER CUPS, SET/6 RED SPOTTY...",(SET/20 RED RETROSPOT PAPER NAPKINS),0.122449,0.132653,0.099490,0.812500,6.125000,0.083247,4.625850
7299,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.102041,0.137755,0.099490,0.975000,7.077778,0.085433,34.489796
12,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
13,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
486,"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...",(ALARM CLOCK BAKELIKE PINK),0.079082,0.102041,0.063776,0.806452,7.903226,0.055706,4.639456
487,"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...",(ALARM CLOCK BAKELIKE RED),0.073980,0.094388,0.063776,0.862069,9.133271,0.056793,6.565689


<p> No caso, quanto maior o suporte, em conjunto de uma boa confiança e um bom "lift" serão os produtos que mais desejamos associar, por isso, escolhemos esta ordem, e por isso, produtos da lista acima são boas associações 
   

<h2> Faça o mesmo procedimento para a Alemanha
(Country=’Germany’). Existe alguma similaridade nas principais
regras? Quais produtos você recomendaria colocar próximos se
você fosse organizar as prateleiras de um supermercado físico?

In [115]:
df = df[df['Country'] == 'Germany']
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,2010-12-01 13:04:00,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,2010-12-01 13:04:00,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,2010-12-01 13:04:00,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,2010-12-01 13:04:00,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,2010-12-01 13:04:00,1.95,12662.0,Germany
...,...,...,...,...,...,...,...,...
541801,581578,22993,SET OF 4 PANTRY JELLY MOULDS,12,2011-12-09 12:16:00,1.25,12713.0,Germany
541802,581578,22907,PACK OF 20 NAPKINS PANTRY DESIGN,12,2011-12-09 12:16:00,0.85,12713.0,Germany
541803,581578,22908,PACK OF 20 NAPKINS RED APPLES,12,2011-12-09 12:16:00,0.85,12713.0,Germany
541804,581578,23215,JINGLE BELL HEART ANTIQUE SILVER,12,2011-12-09 12:16:00,2.08,12713.0,Germany


In [116]:
df = df[df['Country'] == 'Germany']
df_grouped = df.groupby(['InvoiceNo','Description'])['Quantity'].sum()

cestaAlemanha = df_grouped.unstack(fill_value=0)
cestaAlemanha = cestaAlemanha.reset_index().set_index('InvoiceNo')
cestaAlemanha

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,...,YULETIDE IMAGES GIFT WRAP SET,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536840,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536983,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581266,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581494,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [117]:
cestaAlemanha = cestaAlemanha.applymap(encode_units)
cestaAlemanha.drop('POSTAGE',inplace=True,axis=1)
cestaAlemanha

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,...,YULETIDE IMAGES GIFT WRAP SET,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536840,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536861,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536967,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536983,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581266,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
581494,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
581570,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [118]:
frequent_itemsets = apriori(cestaAlemanha, min_support=0.01,use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.024070,(10 COLOUR SPACEBOY PEN)
1,0.021882,(12 PENCIL SMALL TUBE WOODLAND)
2,0.013129,(12 PENCILS SMALL TUBE RED RETROSPOT)
3,0.019694,(12 PENCILS TALL TUBE WOODLAND)
4,0.032823,(3 HOOK HANGER MAGIC GARDEN)
...,...,...
3966,0.010941,"(RED SPOT CERAMIC DRAWER KNOB, WHITE SPOT BLUE..."
3967,0.010941,"(SPACEBOY LUNCH BOX, RED HARMONICA IN BOX, RE..."
3968,0.010941,"(RED SPOT CERAMIC DRAWER KNOB, WHITE SPOT BLUE..."
3969,0.010941,"(RED SPOT CERAMIC DRAWER KNOB, WHITE SPOT BLUE..."


In [127]:
regrasAlemanha = association_rules(frequent_itemsets,metric="confidence", min_threshold=0.7)
regrasAlemanha

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(12 PENCILS TALL TUBE WOODLAND),(ROUND SNACK BOXES SET OF4 WOODLAND),0.019694,0.245077,0.015317,0.777778,3.173611,0.010491,3.397155
1,(BRIGHT BLUES RIBBONS),(6 RIBBONS RUSTIC CHARM),0.010941,0.102845,0.010941,1.000000,9.723404,0.009816,inf
2,(OWL DOORSTOP),(6 RIBBONS RUSTIC CHARM),0.015317,0.102845,0.010941,0.714286,6.945289,0.009366,3.140044
3,(ROMANTIC PINKS RIBBONS),(6 RIBBONS RUSTIC CHARM),0.021882,0.102845,0.015317,0.700000,6.806383,0.013067,2.990518
4,(60 CAKE CASES DOLLY GIRL DESIGN),(PACK OF 72 RETROSPOT CAKE CASES),0.015317,0.085339,0.013129,0.857143,10.043956,0.011822,6.402626
...,...,...,...,...,...,...,...,...,...
3957,"(BLUE SPOT CERAMIC DRAWER KNOB, WHITE SPOT BLU...","(WHITE SPOT RED CERAMIC DRAWER KNOB, ROUND SNA...",0.015317,0.019694,0.010941,0.714286,36.269841,0.010639,3.431072
3958,"(ROUND SNACK BOXES SET OF4 WOODLAND, WHITE SPO...","(WHITE SPOT RED CERAMIC DRAWER KNOB, RED SPOT ...",0.013129,0.021882,0.010941,0.833333,38.083333,0.010654,5.868709
3959,"(BLUE SPOT CERAMIC DRAWER KNOB, ROUND SNACK BO...","(WHITE SPOT RED CERAMIC DRAWER KNOB, RED SPOT ...",0.013129,0.017505,0.010941,0.833333,47.604167,0.010711,5.894967
3960,"(WHITE SPOT RED CERAMIC DRAWER KNOB, ROUND SNA...","(RED SPOT CERAMIC DRAWER KNOB, RED STRIPE CERA...",0.013129,0.015317,0.010941,0.833333,54.404762,0.010740,5.908096


<h3> a) Qual a regra com maior suporte?


In [120]:
frequent_itemsets.sort_values(by='support',ascending=False).head(1)

,support,itemsets
387,0.245077,(ROUND SNACK BOXES SET OF4 WOODLAND)


<p> A regra com maios suporte é do seguinte itemset: (ROUND SNACK BOXES SET OF4 WOODLAND)

<h3> b) Qual a regra com maior confiança?


In [128]:
regrasAlemanha.sort_values(by='confidence',ascending=False).head(1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1365,"(POPPY'S PLAYHOUSE BATHROOM, POPPY'S PLAYHOUSE...",(POPPY'S PLAYHOUSE LIVINGROOM),0.010941,0.013129,0.010941,1.0,76.166667,0.010797,inf


<h3>c) Qual a regra com maior lift?


In [129]:
regrasAlemanha.sort_values(by='lift',ascending=False).head(1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3135,"(ROUND STORAGE TIN VINTAGE LEAF, SET OF 4 KNIC...","(STORAGE TIN VINTAGE LEAF, STORAGE TIN VINTAGE...",0.010941,0.010941,0.010941,1.0,91.4,0.010821,inf


<h3> d) Quantas regras possuem suporte maior ou igual a 0.02 e
confiança maior ou igual a 0.8?

In [132]:
len(regrasAlemanha[(regras["support"] >= 0.02) & (regrasAlemanha["confidence"] >= 0.8)])

56

<h3> e) Quantas regras possuem lift maior ou igual a 4 e confiança
maior ou igual a 0.8?

In [131]:
len(regrasAlemanha[(regras["lift"] >= 4) & (regrasAlemanha["confidence"] >= 0.8)])

2537

<h3> f) Quais produtos você recomendaria colocar próximos se você
fosse organizar um supermercado físico?

In [133]:
regrasAlemanha[(regrasAlemanha["lift"] >= 4) & (regrasAlemanha["confidence"] >= 0.8)].sort_values(by='support',ascending=False).head(30)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
148,(RED RETROSPOT CHARLOTTE BAG),(WOODLAND CHARLOTTE BAG),0.070022,0.126915,0.059081,0.843750,6.648168,0.050194,5.587746
182,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.052516,0.056893,0.045952,0.875000,15.379808,0.042964,7.544858
183,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.056893,0.052516,0.045952,0.807692,15.379808,0.042964,4.926915
41,(CHILDRENS CUTLERY SPACEBOY),(CHILDRENS CUTLERY DOLLY GIRL),0.048140,0.050328,0.039387,0.818182,16.256917,0.036965,5.223195
184,(SPACEBOY CHILDRENS BOWL),(SPACEBOY CHILDRENS CUP),0.041575,0.043764,0.037199,0.894737,20.444737,0.035380,9.084245
185,(SPACEBOY CHILDRENS CUP),(SPACEBOY CHILDRENS BOWL),0.043764,0.041575,0.037199,0.850000,20.444737,0.035380,6.389497
38,(CHARLOTTE BAG SUKI DESIGN),(WOODLAND CHARLOTTE BAG),0.045952,0.126915,0.037199,0.809524,6.378489,0.031367,4.583698
51,(COFFEE MUG PEARS DESIGN),(COFFEE MUG APPLES DESIGN),0.039387,0.061269,0.035011,0.888889,14.507937,0.032598,8.448578
78,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.035011,0.078775,0.032823,0.937500,11.901042,0.030065,14.739606
75,(JAM JAR WITH GREEN LID),(JAM JAR WITH PINK LID),0.035011,0.063457,0.032823,0.937500,14.773707,0.030601,14.984683


<p> No caso, quanto maior o suporte, em conjunto de uma boa confiança e um bom "lift" serão os produtos que mais desejamos associar, por isso, escolhemos esta ordem, e por isso, produtos da lista acima são boas associações 
   

<h2> Existe alguma similaridade nas principais
regras?

<p> Pegando as 30 principais regras como referência, há similaridade, como em "(DOLLY GIRL CHILDRENS BOWL)", que aparece em ambos os dataframes (Alemanha = 58 & Franca = 65)

In [159]:
regrasAlemanha[regrasAlemanha["antecedents"].isin(regras[(regras["lift"] >= 4) & (regras["confidence"] >= 0.8)].sort_values(by='support',ascending=False)["antecedents"].head(30)) == True].head(30)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
8,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.032823,0.045952,0.024070,0.733333,15.958730,0.022562,3.577681
41,(CHILDRENS CUTLERY SPACEBOY),(CHILDRENS CUTLERY DOLLY GIRL),0.048140,0.050328,0.039387,0.818182,16.256917,0.036965,5.223195
42,(CHILDRENS CUTLERY DOLLY GIRL),(CHILDRENS CUTLERY SPACEBOY),0.050328,0.048140,0.039387,0.782609,16.256917,0.036965,4.378556
58,(DOLLY GIRL CHILDRENS BOWL),(DOLLY GIRL CHILDRENS CUP),0.026258,0.024070,0.024070,0.916667,38.083333,0.023438,11.711160
104,(PACK OF 6 SKULL PAPER CUPS),(PACK OF 20 SKULL PAPER NAPKINS),0.015317,0.015317,0.010941,0.714286,46.632653,0.010706,3.446389
105,(PACK OF 20 SKULL PAPER NAPKINS),(PACK OF 6 SKULL PAPER CUPS),0.015317,0.015317,0.010941,0.714286,46.632653,0.010706,3.446389
106,(PACK OF 20 SKULL PAPER NAPKINS),(PACK OF 6 SKULL PAPER PLATES),0.015317,0.017505,0.013129,0.857143,48.964286,0.012861,6.877462
107,(PACK OF 6 SKULL PAPER PLATES),(PACK OF 20 SKULL PAPER NAPKINS),0.017505,0.015317,0.013129,0.750000,48.964286,0.012861,3.938731
108,(PACK OF 20 SKULL PAPER NAPKINS),(SET/20 RED RETROSPOT PAPER NAPKINS),0.015317,0.056893,0.010941,0.714286,12.554945,0.010069,3.300875
109,(PACK OF 20 SKULL PAPER NAPKINS),(SET/6 RED SPOTTY PAPER CUPS),0.015317,0.052516,0.010941,0.714286,13.601190,0.010137,3.316193
